In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

# Set random seed for reproducibility
torch.manual_seed(42)

# Load and preprocess the MNIST dataset
train_dataset = MNIST(root='data/', train=True, transform=ToTensor(), download=True)
test_dataset = MNIST(root='data/', train=False, transform=ToTensor())

# Define the model
class PureCNN(nn.Module):
    def __init__(self):
        super(PureCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, kernel_size=3)
        self.conv2 = nn.Conv2d(8, 16, kernel_size=3)
        self.fc = nn.Linear(16 * 5 * 5, 10)

    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.max_pool2d(x, 2)
        x = nn.functional.relu(self.conv2(x))
        x = nn.functional.max_pool2d(x, 2)
        x = x.view(-1, 16 * 5 * 5)
        x = self.fc(x)
        return x

# Instantiate the model
model = PureCNN()

# Count the total number of trainable parameters
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total Trainable Parameters:", total_params)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move the model to the device
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)

# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    train_correct = 0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Calculate training accuracy
        _, predicted = torch.max(outputs.data, 1)
        train_correct += (predicted == labels).sum().item()

        train_loss += loss.item() * images.size(0)

    train_loss /= len(train_loader.dataset)
    train_accuracy = 100.0 * train_correct / len(train_loader.dataset)

    # Validation loop
    model.eval()
    val_loss = 0.0
    val_correct = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            _, predicted = torch.max(outputs.data, 1)
            val_correct += (predicted == labels).sum().item()

            val_loss += loss.item() * images.size(0)

    val_loss /= len(test_loader.dataset)
    val_accuracy = 100.0 * val_correct / len(test_loader.dataset)

    print(f"Epoch [{epoch+1}/{num_epochs}], "
          f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, "
          f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")
    
    # # Check if validation accuracy threshold is reached
    # if val_accuracy >= 99.40:
    #     break


Total Trainable Parameters: 5258
Epoch [1/20], Train Loss: 0.3432, Train Accuracy: 90.19%, Val Loss: 0.1118, Val Accuracy: 96.93%
Epoch [2/20], Train Loss: 0.1049, Train Accuracy: 96.86%, Val Loss: 0.0774, Val Accuracy: 97.68%
Epoch [3/20], Train Loss: 0.0805, Train Accuracy: 97.54%, Val Loss: 0.0641, Val Accuracy: 98.04%
Epoch [4/20], Train Loss: 0.0676, Train Accuracy: 97.89%, Val Loss: 0.0596, Val Accuracy: 98.20%
Epoch [5/20], Train Loss: 0.0603, Train Accuracy: 98.14%, Val Loss: 0.0517, Val Accuracy: 98.44%
Epoch [6/20], Train Loss: 0.0555, Train Accuracy: 98.27%, Val Loss: 0.0456, Val Accuracy: 98.63%
Epoch [7/20], Train Loss: 0.0512, Train Accuracy: 98.39%, Val Loss: 0.0485, Val Accuracy: 98.39%
Epoch [8/20], Train Loss: 0.0473, Train Accuracy: 98.52%, Val Loss: 0.0485, Val Accuracy: 98.53%
Epoch [9/20], Train Loss: 0.0453, Train Accuracy: 98.59%, Val Loss: 0.0466, Val Accuracy: 98.50%
Epoch [10/20], Train Loss: 0.0417, Train Accuracy: 98.71%, Val Loss: 0.0446, Val Accuracy: 98.

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Set random seed for reproducibility
tf.random.set_seed(42)

# Load and preprocess the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1) / 255.0
x_test = x_test.reshape(-1, 28, 28, 1) / 255.0

# Define the model
model = Sequential([
    Conv2D(8, kernel_size=3, activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D(pool_size=2),
    Conv2D(16, kernel_size=3, activation='relu'),
    MaxPooling2D(pool_size=2),
    Flatten(),
    Dense(10, activation='softmax')
])

# Count the total number of trainable parameters
total_params = model.count_params()
print("Total Trainable Parameters:", total_params)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    # Train the model
    model.fit(x_train, y_train, batch_size=64, epochs=1, verbose=2)

    # Evaluate the model
    _, train_accuracy = model.evaluate(x_train, y_train, verbose=0)
    _, val_accuracy = model.evaluate(x_test, y_test, verbose=0)

    print(f"Epoch [{epoch+1}/{num_epochs}], "
          f"Train Accuracy: {train_accuracy:.4f}%, "
          f"Val Accuracy: {val_accuracy:.4f}%")
    

Total Trainable Parameters: 5258
938/938 - 4s - loss: 0.3544 - accuracy: 0.8979 - 4s/epoch - 4ms/step
Epoch [1/20], Train Accuracy: 0.9598%, Val Accuracy: 0.9622%
938/938 - 3s - loss: 0.1108 - accuracy: 0.9664 - 3s/epoch - 3ms/step
Epoch [2/20], Train Accuracy: 0.9730%, Val Accuracy: 0.9751%
938/938 - 3s - loss: 0.0829 - accuracy: 0.9749 - 3s/epoch - 3ms/step
Epoch [3/20], Train Accuracy: 0.9781%, Val Accuracy: 0.9783%
938/938 - 3s - loss: 0.0694 - accuracy: 0.9787 - 3s/epoch - 3ms/step
Epoch [4/20], Train Accuracy: 0.9810%, Val Accuracy: 0.9814%
938/938 - 3s - loss: 0.0610 - accuracy: 0.9816 - 3s/epoch - 3ms/step
Epoch [5/20], Train Accuracy: 0.9825%, Val Accuracy: 0.9816%
938/938 - 3s - loss: 0.0548 - accuracy: 0.9837 - 3s/epoch - 3ms/step
Epoch [6/20], Train Accuracy: 0.9836%, Val Accuracy: 0.9826%
938/938 - 3s - loss: 0.0499 - accuracy: 0.9848 - 3s/epoch - 3ms/step
Epoch [7/20], Train Accuracy: 0.9849%, Val Accuracy: 0.9834%
938/938 - 3s - loss: 0.0459 - accuracy: 0.9862 - 3s/epoch

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

# Set random seed for reproducibility
torch.manual_seed(42)

# Load and preprocess the MNIST dataset
train_dataset = MNIST(root='data/', train=True, transform=ToTensor(), download=True)
test_dataset = MNIST(root='data/', train=False, transform=ToTensor())

# Define the model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(8, 16, kernel_size=3, padding=1)
        self.fc = nn.Linear(16 * 7 * 7, 10)

    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = self.pool(x)
        x = nn.functional.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(-1, 16 * 7 * 7)
        x = self.fc(x)
        return x

# Instantiate the model
model = SimpleCNN()

# Count the total number of trainable parameters
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total Trainable Parameters:", total_params)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move the model to the device
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)

# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    train_correct = 0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Calculate training accuracy
        _, predicted = torch.max(outputs.data, 1)
        train_correct += (predicted == labels).sum().item()

        train_loss += loss.item() * images.size(0)

    train_loss /= len(train_loader.dataset)
    train_accuracy = 100.0 * train_correct / len(train_loader.dataset)

    # Validation loop
    model.eval()
    val_loss = 0.0
    val_correct = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            _, predicted = torch.max(outputs.data, 1)
            val_correct += (predicted == labels).sum().item()

            val_loss += loss.item() * images.size(0)

    val_loss /= len(test_loader.dataset)
    val_accuracy = 100.0 * val_correct / len(test_loader.dataset)

    print(f"Epoch [{epoch+1}/{num_epochs}], "
          f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}%, "
          f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}%")


Total Trainable Parameters: 9098
Epoch [1/20], Train Loss: 0.3215, Train Accuracy: 90.7600%, Val Loss: 0.1041, Val Accuracy: 96.7900%
Epoch [2/20], Train Loss: 0.1005, Train Accuracy: 97.0533%, Val Loss: 0.0678, Val Accuracy: 97.7500%
Epoch [3/20], Train Loss: 0.0764, Train Accuracy: 97.6617%, Val Loss: 0.0604, Val Accuracy: 98.0600%
Epoch [4/20], Train Loss: 0.0628, Train Accuracy: 98.0983%, Val Loss: 0.0533, Val Accuracy: 98.3900%
Epoch [5/20], Train Loss: 0.0545, Train Accuracy: 98.3450%, Val Loss: 0.0498, Val Accuracy: 98.4100%
Epoch [6/20], Train Loss: 0.0470, Train Accuracy: 98.6067%, Val Loss: 0.0451, Val Accuracy: 98.5900%
Epoch [7/20], Train Loss: 0.0427, Train Accuracy: 98.6650%, Val Loss: 0.0376, Val Accuracy: 98.8100%
Epoch [8/20], Train Loss: 0.0385, Train Accuracy: 98.8450%, Val Loss: 0.0422, Val Accuracy: 98.6400%
Epoch [9/20], Train Loss: 0.0358, Train Accuracy: 98.9017%, Val Loss: 0.0412, Val Accuracy: 98.7200%
Epoch [10/20], Train Loss: 0.0327, Train Accuracy: 99.0017

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

# Set random seed for reproducibility
torch.manual_seed(42)

# Load and preprocess the MNIST dataset
train_dataset = MNIST(root='data/', train=True, transform=ToTensor(), download=True)
test_dataset = MNIST(root='data/', train=False, transform=ToTensor())

# Define the model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(8, 16, kernel_size=3, padding=1)
        self.fc = nn.Linear(16 * 7 * 7, 10)

    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = self.pool(x)
        x = nn.functional.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(-1, 16 * 7 * 7)
        x = self.fc(x)
        return x

# Instantiate the model
model = SimpleCNN()

# Count the total number of trainable parameters
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total Trainable Parameters:", total_params)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move the model to the device
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    train_correct = 0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Calculate training accuracy
        _, predicted = torch.max(outputs.data, 1)
        train_correct += (predicted == labels).sum().item()

        train_loss += loss.item() * images.size(0)

    train_loss /= len(train_loader.dataset)
    train_accuracy = 100.0 * train_correct / len(train_loader.dataset)

    # Validation loop
    model.eval()
    val_loss = 0.0
    val_correct = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            _, predicted = torch.max(outputs.data, 1)
            val_correct += (predicted == labels).sum().item()

            val_loss += loss.item() * images.size(0)

    val_loss /= len(test_loader.dataset)
    val_accuracy = 100.0 * val_correct / len(test_loader.dataset)

    print(f"Epoch [{epoch+1}/{num_epochs}], "
          f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, "
          f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

# Set random seed for reproducibility
torch.manual_seed(42)

# Load and preprocess the MNIST dataset
train_dataset = MNIST(root='data/', train=True, transform=ToTensor(), download=True)
test_dataset = MNIST(root='data/', train=False, transform=ToTensor())

# Define the model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.pool = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.fc = nn.Linear(32 * 7 * 7, 10)

    def forward(self, x):
        x = nn.functional.relu(self.bn1(self.conv1(x)))
        x = self.pool(x)
        x = nn.functional.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)
        x = x.view(-1, 32 * 7 * 7)
        x = self.fc(x)
        return x

# Instantiate the model
model = SimpleCNN()

# Count the total number of trainable parameters
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total Trainable Parameters:", total_params)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move the model to the device
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    train_correct = 0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Calculate training accuracy
        _, predicted = torch.max(outputs.data, 1)
        train_correct += (predicted == labels).sum().item()

        train_loss += loss.item() * images.size(0)

    train_loss /= len(train_loader.dataset)
    train_accuracy = 100.0 * train_correct / len(train_loader.dataset)

    # Validation loop
    model.eval()
    val_loss = 0.0
    val_correct = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            _, predicted = torch.max(outputs.data, 1)
            val_correct += (predicted == labels).sum().item()

            val_loss += loss.item() * images.size(0)

    val_loss /= len(test_loader.dataset)
    val_accuracy = 100.0 * val_correct / len(test_loader.dataset)

    print(f"Epoch [{epoch+1}/{num_epochs}], "
          f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}%, "
          f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}%")


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

# Set random seed for reproducibility
torch.manual_seed(42)

# Load and preprocess the MNIST dataset
train_dataset = MNIST(root='data/', train=True, transform=ToTensor(), download=True)
test_dataset = MNIST(root='data/', train=False, transform=ToTensor())

# Define the model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 4, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(4)
        self.pool = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(4, 8, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(8)
        self.fc = nn.Linear(8 * 7 * 7, 10)

    def forward(self, x):
        x = nn.functional.relu(self.bn1(self.conv1(x)))
        x = self.pool(x)
        x = nn.functional.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)
        x = x.view(-1, 8 * 7 * 7)
        x = self.fc(x)
        return x

# Instantiate the model
model = SimpleCNN()

# Count the total number of trainable parameters
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total Trainable Parameters:", total_params)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move the model to the device
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    train_correct = 0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Calculate training accuracy
        _, predicted = torch.max(outputs.data, 1)
        train_correct += (predicted == labels).sum().item()

        train_loss += loss.item() * images.size(0)

    train_loss /= len(train_loader.dataset)
    train_accuracy = 100.0 * train_correct / len(train_loader.dataset)

    # Validation loop
    model.eval()
    val_loss = 0.0
    val_correct = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            _, predicted = torch.max(outputs.data, 1)
            val_correct += (predicted == labels).sum().item()

            val_loss += loss.item() * images.size(0)

    val_loss /= len(test_loader.dataset)
    val_accuracy = 100.0 * val_correct / len(test_loader.dataset)

    print(f"Epoch [{epoch+1}/{num_epochs}], "
          f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}%, "
          f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}%")


Total Trainable Parameters: 4290
Epoch [1/50], Train Loss: 0.2273, Train Accuracy: 93.7017%, Val Loss: 0.0927, Val Accuracy: 97.1800%
Epoch [2/50], Train Loss: 0.0790, Train Accuracy: 97.5933%, Val Loss: 0.0714, Val Accuracy: 97.6500%
Epoch [3/50], Train Loss: 0.0658, Train Accuracy: 97.9617%, Val Loss: 0.0539, Val Accuracy: 98.3000%
Epoch [4/50], Train Loss: 0.0586, Train Accuracy: 98.1867%, Val Loss: 0.0521, Val Accuracy: 98.3900%
Epoch [5/50], Train Loss: 0.0530, Train Accuracy: 98.3950%, Val Loss: 0.0584, Val Accuracy: 98.2800%
Epoch [6/50], Train Loss: 0.0496, Train Accuracy: 98.4733%, Val Loss: 0.0535, Val Accuracy: 98.3200%
Epoch [7/50], Train Loss: 0.0470, Train Accuracy: 98.5367%, Val Loss: 0.0481, Val Accuracy: 98.4500%
Epoch [8/50], Train Loss: 0.0442, Train Accuracy: 98.6000%, Val Loss: 0.0482, Val Accuracy: 98.4300%
Epoch [9/50], Train Loss: 0.0430, Train Accuracy: 98.6167%, Val Loss: 0.0465, Val Accuracy: 98.5600%
Epoch [10/50], Train Loss: 0.0400, Train Accuracy: 98.7083

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization

# Set random seed for reproducibility
tf.random.set_seed(42)

# Load and preprocess the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1) / 255.0
x_test = x_test.reshape(-1, 28, 28, 1) / 255.0

# Define the model
model = Sequential([
    Conv2D(4, kernel_size=3, padding='same', activation='relu', input_shape=(28, 28, 1)),
    BatchNormalization(),
    MaxPooling2D(pool_size=2),
    Conv2D(8, kernel_size=3, padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=2),
    Flatten(),
    Dense(10, activation='softmax')
])

# Count the total number of trainable parameters
total_params = model.count_params()
print("Total Trainable Parameters:", total_params)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    # Train the model
    model.fit(x_train, y_train, batch_size=32, epochs=1, verbose=2)

    # Evaluate the model
    _, train_accuracy = model.evaluate(x_train, y_train, verbose=0)
    _, val_accuracy = model.evaluate(x_test, y_test, verbose=0)

    print(f"Epoch [{epoch+1}/{num_epochs}], "
          f"Train Accuracy: {train_accuracy:.4f}%, "
          f"Val Accuracy: {val_accuracy:.4f}%")


Total Trainable Parameters: 4314
1875/1875 - 8s - loss: 0.2231 - accuracy: 0.9305 - 8s/epoch - 4ms/step
Epoch [1/50], Train Accuracy: 0.9695%, Val Accuracy: 0.9668%
1875/1875 - 6s - loss: 0.0834 - accuracy: 0.9738 - 6s/epoch - 3ms/step
Epoch [2/50], Train Accuracy: 0.9745%, Val Accuracy: 0.9697%
1875/1875 - 6s - loss: 0.0677 - accuracy: 0.9790 - 6s/epoch - 3ms/step
Epoch [3/50], Train Accuracy: 0.9773%, Val Accuracy: 0.9726%
1875/1875 - 6s - loss: 0.0588 - accuracy: 0.9819 - 6s/epoch - 3ms/step
Epoch [4/50], Train Accuracy: 0.9808%, Val Accuracy: 0.9766%
1875/1875 - 5s - loss: 0.0532 - accuracy: 0.9837 - 5s/epoch - 3ms/step
Epoch [5/50], Train Accuracy: 0.9822%, Val Accuracy: 0.9772%
1875/1875 - 6s - loss: 0.0492 - accuracy: 0.9850 - 6s/epoch - 3ms/step
Epoch [6/50], Train Accuracy: 0.9824%, Val Accuracy: 0.9771%
1875/1875 - 6s - loss: 0.0459 - accuracy: 0.9861 - 6s/epoch - 3ms/step
Epoch [7/50], Train Accuracy: 0.9847%, Val Accuracy: 0.9779%
1875/1875 - 6s - loss: 0.0430 - accuracy: 0

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization

# Set random seed for reproducibility
tf.random.set_seed(42)

# Load and preprocess the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1) / 255.0
x_test = x_test.reshape(-1, 28, 28, 1) / 255.0

# Define the model
model = Sequential([
    Conv2D(2, kernel_size=3, padding='same', activation='relu', input_shape=(28, 28, 1)),
    BatchNormalization(),
    MaxPooling2D(pool_size=2),
    Conv2D(4, kernel_size=3, padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=2),
    Flatten(),
    Dense(10, activation='softmax')
])

# Count the total number of trainable parameters
total_params = model.count_params()
print("Total Trainable Parameters:", total_params)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    # Train the model
    model.fit(x_train, y_train, batch_size=32, epochs=1, verbose=2)

    # Evaluate the model
    _, train_accuracy = model.evaluate(x_train, y_train, verbose=0)
    _, val_accuracy = model.evaluate(x_test, y_test, verbose=0)

    print(f"Epoch [{epoch+1}/{num_epochs}], "
          f"Train Accuracy: {train_accuracy:.4f}%, "
          f"Val Accuracy: {val_accuracy:.4f}%")


Total Trainable Parameters: 2090
1875/1875 - 8s - loss: 0.3861 - accuracy: 0.8813 - 8s/epoch - 4ms/step
Epoch [1/50], Train Accuracy: 0.9516%, Val Accuracy: 0.9533%
1875/1875 - 6s - loss: 0.1345 - accuracy: 0.9591 - 6s/epoch - 3ms/step
Epoch [2/50], Train Accuracy: 0.9653%, Val Accuracy: 0.9650%
1875/1875 - 6s - loss: 0.1073 - accuracy: 0.9668 - 6s/epoch - 3ms/step
Epoch [3/50], Train Accuracy: 0.9693%, Val Accuracy: 0.9682%
1875/1875 - 5s - loss: 0.0968 - accuracy: 0.9702 - 5s/epoch - 3ms/step
Epoch [4/50], Train Accuracy: 0.9720%, Val Accuracy: 0.9685%
1875/1875 - 5s - loss: 0.0909 - accuracy: 0.9722 - 5s/epoch - 3ms/step
Epoch [5/50], Train Accuracy: 0.9722%, Val Accuracy: 0.9691%
1875/1875 - 5s - loss: 0.0868 - accuracy: 0.9735 - 5s/epoch - 3ms/step
Epoch [6/50], Train Accuracy: 0.9728%, Val Accuracy: 0.9699%
1875/1875 - 5s - loss: 0.0837 - accuracy: 0.9741 - 5s/epoch - 3ms/step
Epoch [7/50], Train Accuracy: 0.9728%, Val Accuracy: 0.9691%
1875/1875 - 5s - loss: 0.0813 - accuracy: 0